In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [3]:
def elastic_search(query, course):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    #result_docs = []
    
    #for hit in response['hits']['hits']:
     #   result_docs.append(hit['_source'])
    
    return response

In [4]:
from pprint import pprint

search_result = elastic_search("How do copy a file to a Docker container?", "machine-learning-zoomcamp")

pprint(search_result['hits']['hits'])

[{'_id': '_P_aVZcBJ-Mqo_mn84Cr',
  '_index': 'course-questions',
  '_score': 73.38676,
  '_source': {'course': 'machine-learning-zoomcamp',
              'question': 'How do I debug a docker container?',
              'section': '5. Deploying Machine Learning Models',
              'text': 'Launch the container image in interactive mode and '
                      'overriding the entrypoint, so that it starts a bash '
                      'command.\n'
                      'docker run -it --entrypoint bash <image>\n'
                      'If the container is already running, execute a command '
                      'in the specific container:\n'
                      'docker ps (find the container-id)\n'
                      'docker exec -it <container-id> bash\n'
                      '(Marcos MJD)'}},
 {'_id': 'G__aVZcBJ-Mqo_mn9IE3',
  '_index': 'course-questions',
  '_score': 66.688705,
  '_source': {'course': 'machine-learning-zoomcamp',
              'question': 'How do I copy

In [5]:
def build_prompt(question, search_result):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    for hit in search_result["hits"]["hits"]:
        context = context + "\n\n" + context_template.format(question=hit["_source"]["question"], text=hit["_source"]["text"])

    prompt = prompt_template.format(question=question, context=context)

    return prompt

In [ ]:
print(build_prompt("How do copy a file to a Docker container?", search_result))

In [6]:
print(len(build_prompt("How do copy a file to a Docker container?", search_result)))

1448


In [7]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [11]:
our_prompt = build_prompt("How do copy a file to a Docker container?", search_result)
enc = tiktoken.encoding_for_model("gpt-4o")
enc_result = enc.encode(our_prompt)

In [13]:
len(enc_result)

320

In [14]:
from openai import OpenAI

llm_client = OpenAI()


def llm(prompt):
    response = llm_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
result = llm(our_prompt)

In [16]:
result

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\n\nThis command allows you to copy files from your local machine into a running Docker container.'